In [1]:
import os

In [2]:
%pwd

'/Users/ravina/Desktop/CustomerChurnPrediction/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/ravina/Desktop/CustomerChurnPrediction'

In [5]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)  #this is not python class but dataclass, here you can define the veriables without using self keyword.
class DataTransformationConfig:
    root_dir:Path
    data_path: Path

In [6]:
from CustomerChurnPrediction.constants import *
from CustomerChurnPrediction.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,

        )

        return data_transformation_config

In [8]:
import os
from CustomerChurnPrediction import logger
from sklearn.model_selection import train_test_split
import pandas as pd 
from sklearn.preprocessing import StandardScaler
import imblearn
from imblearn.over_sampling import RandomOverSampler



In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_splitting(self):
        df = pd.read_csv(self.config.data_path)
        
        # Separate the target variable 'Exited'
        y = df['Exited']
        X=df.drop(['RowNumber', 'CustomerId', 'Surname', 'Complain','Geography', 'Gender', 'Card Type','Exited',], axis=1)
        print("columns of X:",X.columns)
        
        cat_columns = ['Geography', 'Gender', 'Card Type']
        num_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',"HasCrCard",'IsActiveMember','EstimatedSalary', 'Satisfaction Score', 'Point Earned']
        
        # Handle categorical variables using one-hot encoding
        df_cat_encoded = pd.get_dummies(df[cat_columns], dtype=int,drop_first=True)

        # Standard scaling on numerical columns
        scaler = StandardScaler()
        df_scaled = scaler.fit_transform(X[num_columns])
        df_scaled = pd.DataFrame(df_scaled, columns=num_columns)

        # Concatenate the encoded categorical, scaled numerical and target y.
        df_transformed = pd.concat([df_scaled,df_cat_encoded, y], axis=1)

        # Split into training and testing sets
        train, test = train_test_split(df_transformed, test_size=0.25, random_state=42)
        
        # Apply oversampling to training set
        ros = RandomOverSampler(random_state=0)
        X_train, y_train = ros.fit_resample(train.drop(columns=['Exited']), train['Exited'])
        train_resampled = pd.concat([pd.DataFrame(X_train, columns=train.drop(columns=['Exited']).columns), pd.DataFrame(y_train, columns=['Exited'])], axis=1)
        print("columns of train_resample:",train_resampled.columns)
        # Save train and test sets to CSV
        train_resampled.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
        test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

        logger.info("Data has been split into training and test sets.")
        logger.info(f"Training set shape: {train_resampled.shape}")
        logger.info(f"Test set shape: {test.shape}")




In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e


[2024-03-19 18:39:12,860:INFO:yaml file:config/config.yaml loaded successfully]
[2024-03-19 18:39:12,862:INFO:yaml file:params.yaml loaded successfully]
[2024-03-19 18:39:12,865:INFO:yaml file:Schema.yaml loaded successfully]
[2024-03-19 18:39:12,866:INFO:created directory at:artifacts]
[2024-03-19 18:39:12,866:INFO:created directory at:artifacts/data_transformation]
columns of X: Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Satisfaction Score',
       'Point Earned'],
      dtype='object')
columns of train_resample: Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Satisfaction Score',
       'Point Earned', 'Geography_Germany', 'Geography_Spain', 'Gender_Male',
       'Card Type_GOLD', 'Card Type_PLATINUM', 'Card Type_SILVER', 'Exited'],
      dtype='object')
[2024-03-19 18:39:13,111:INFO:Data has been split into training and test 